# Import dependencies

In [1]:
import time
import operator as optr
from collections import Counter
startTime=time.time() # to calculate total execution time

# Read given files to find misspelled word

In [2]:
with open('./jang_errors.txt', 'r', encoding='utf8', errors='ignore') as f:
    erorrsFile = f.readlines() # wrongly spelled file
with open('./jang_nonerrors.txt', 'r', encoding='utf8', errors='ignore') as f:
    correctedFile = f.readlines() # correctly spelled file
with open('./wordlist.txt', 'r', encoding='utf8', errors='ignore') as f:
    wordsFile = f.readlines() # list of valid urdu words, dictionary
wordsList=list()
for word in wordsFile:
    wordsList.append(word[:-1])

misspelledWords=[]
correctSpelledWords=[]
misspelledPrevious=dict()
no_of_lines = len(correctedFile) # or len(erorrsFile) # as no of lines are equal
#print(no_of_lines)
print('Word Index\tWrong word\tCorrect word\tPrevious word')
misspell_count=0
for idx in range(no_of_lines):
    errLine = erorrsFile[idx].replace('<s>', '').replace('</s>', '')
    correctLine = correctedFile[idx].replace('<s>', '').replace('</s>', '')
    errLineWords = errLine.split(' ')
    correctLineWords = correctLine.split(' ')
    errLineWords.remove('')
    correctLineWords.remove('')
    #print(errLine)
    #print(correctLine)
    no_of_words = -1
    if (len(correctLineWords) == len(errLineWords)):
        no_of_words = len(errLineWords) #or len(correctLineWords) # as no of words are equal
    else:
        print ('Error line and correct line have word count difference..Abort!!')
        break
    for word_idx in range(no_of_words):
        if errLineWords[word_idx] != correctLineWords[word_idx] and errLineWords[word_idx] not in wordsList and errLineWords[word_idx] != '' and correctLineWords[word_idx] != '':
            misspelledWords.append(errLineWords[word_idx])
            correctSpelledWords.append(correctLineWords[word_idx])
            misspelledPrevious[errLineWords[word_idx]] = errLineWords[word_idx-1]
            print(misspell_count, '\t\t',errLineWords[word_idx], '\t\t', correctLineWords[word_idx], '\t\t', misspelledPrevious[errLineWords[word_idx]])
            misspell_count+=1
print('\nTotal misspelled word count', misspell_count)

Word Index	Wrong word	Correct word	Previous word
0 		 ملتن 		 ملتان 		 ڈی
1 		 ہلک 		 ہلاک 		 گرد
2 		 بجکہ 		 جبکہ 		 کردیئے
3 		 اسحلہ 		 اسلحہ 		 قبضےسے
4 		 متام 		 تمام 		 ہے
5 		 ملتن 		 ملتان 		 ڈی
6 		 اتلاع 		 اطلاع 		 خفیہ
7 		 تحصل 		 تحصیل 		 کی
8 		 جھاپا 		 چھاپا 		 پر
9 		 جوبی 		 جوابی 		 کی
10 		 بجکہ 		 جبکہ 		 ہلاک
11 		 قضے 		 قبضے 		 کے
12 		 پاکسان 		 پاکستان 		 طالبان
13 		 شنانہ 		 نشانہ 		 دفاترکو
14 		 جھاپے 		 چھاپے 		 لئے
15 		 ماارے 		 مارے 		 جھاپے
16 		 اچلاس 		 اجلاس 		 مشترکہ
17 		 کطاب 		 خطاب 		 سے
18 		 کطاب 		 خطاب 		 سے
19 		 کطاب 		 خطاب 		 اپنے
20 		 کطاب 		 خطاب 		 تیسرا
21 		 کطاب 		 خطاب 		 اپنے
22 		 اچی 		 اچھی 		 باہمی
23 		 فورغ 		 فروغ 		 کے
24 		 اادہ 		 اعادہ 		 کا
25 		 بتہری 		 بہتری 		 کی
26 		 قللت 		 قلت 		 کی
27 		 رنما 		 رہنما 		 سے
28 		 منبی 		 مبنی 		 پر
29 		 کطاب 		 خطاب 		 کے
30 		 متشرکہ 		 مشترکہ 		 کا
31 		 مں 		 میں 		 آباد
32 		 مں 		 میں 		 اجلاس
33 		 اپوزشن 		 اپوزیشن 		 اراکین
34 		 ردمیان 		 درمیان 		 کے
35 		 اص

# Find all candidate words for all misspelled words 

In [3]:
# Finding candidates for a word
def makeCandidates(word):
    candidates=list()
    # urdu alphabet set from wikipedia 
    #ا ب پ ت ٹ ث ج چ ح خ د ڈ ذ ر ڑ ز ژ س ش ص ض ط ظ ع غ ف ق ک گ ل م ن (ں) و ہ (ھ) ء ی ے
    urdu_charset='ابپتٹثجچحخدڈذرڑزژسشصضطظعغفقکگلمنںوہھءیے' # urdu charset
    for char in urdu_charset:
        #insertion candidates
        for i in range(len(word)+1):
            candidates.append(word[0:i]+char+word[i:]) 
        #substitution candidates
        for i in range(len(word)):
            candidates.append(word[0:i]+char+word[i+1:])
    #deletion candidates
    for i in range(len(word)):
        candidates.append(word[0:i]+word[i+1:])
    #transpose candidates
    if(len(word)>1):
        for i in range(len(word)-1):
            candidates.append(word[0:i]+word[i+1]+word[i]+word[i+2:])
    return candidates

In [4]:
# All candidates for all error words
def getAllCandidates():
    candidatesFirstEdit=list()
    candidatesSecondEdit=list()
    for errWord in misspelledWords:
        #print(errWord)
        candidatesFirstEdit+=makeCandidates(errWord)
    for firstEditCandidate in candidatesFirstEdit:
        #print(firstEditCandidate)
        candidatesSecondEdit+=makeCandidates(firstEditCandidate)
    candidates=set(candidatesFirstEdit).union(set(candidatesSecondEdit)) # removing duplicates
    candidates=set(wordsList).intersection(candidates) # removing invalid candidates
    #print(len(candidatesFirstEdit))
    #print(len(candidatesSecondEdit))
    #print(len(candidates))
    #candidatesFirstEdit:
    #candidatesSecondEdit:
    #candidates
    return candidates
candidates=getAllCandidates()

# Calculate minimum edit distance of error word from candidate word

In [5]:
def calculateMinimumEditDistance(str1, str2):
    #str1: errWord, str2: candidateWord
    #insertion, deletion, substitution costs are 1
    ic, dc, sc = 1, 1, 1
    n, m=len(str1), len(str2)
    MED_DP=[[0 for x in range(m + 1)] for x in range(n + 1)] # initialize empty(zero) matrix for results of subproblems 
    for i in range(1,n+1):
        MED_DP[i][0]=MED_DP[i-1][0]+dc
    for i in range(1,m+1):
        MED_DP[0][i]=MED_DP[0][i-1]+dc  
    for i in range(1,n+1):
        for j in range(1,m+1):
            if (i>1 and j>1 and (str1[i-1]==str2[j-2]) and (str1[i-2]==str2[j-1])):
                MED_DP[i][j]=MED_DP[i-2][j-2]+sc
            elif(str1[i-1]==str2[j-1]):
                MED_DP[i][j]=min([MED_DP[i-1][j]+dc,MED_DP[i-1][j-1]+0,MED_DP[i][j-1]+ic])
            else:
                MED_DP[i][j]=min([MED_DP[i-1][j]+dc,MED_DP[i-1][j-1]+sc,MED_DP[i][j-1]+ic])
    #print(MED_DP)
    return MED_DP[n][m]

# Get candidate words within a given edit distance

In [6]:
def getCandidateWords(err_word, med):
    candidateWords=list()
    for word in candidates:
        MED=calculateMinimumEditDistance(err_word, word)
        if MED < med:
            candidateWords.append(word)
    return candidateWords

# Get tokens from training set for model training

In [7]:
def getTrainingSetTokens():
    with open('./jang.txt', 'r', encoding='utf8', errors='ignore') as f:
        tokens=[]
        for line in f.readlines(): # training set
            tokens+=line.split()
        return tokens

# Make all combinational phrases of adjacent words upto n length 

In [8]:
def makeNGram(n, tokens):
    tokenlen=len(tokens)
    nGramList=[]
    for idx, token in enumerate(tokens):
        singleNGramList=[]
        for i in range(n):
            if(idx+n<=tokenlen):
                singleNGramList.append(tokens[idx+i])
                #print(single_ngramlist)
        if(len(singleNGramList)==n):
            nGramList.append(tuple(singleNGramList))
            #print(single_ngramlist)
            #print(nGramList)
    return nGramList

# Calculate count of unigrams and bigrams

In [9]:
def calculateCounts():
    tokens=getTrainingSetTokens()
    unigramList=makeNGram(1, tokens)
    bigramList=makeNGram(2, tokens)
    #print(unigramList)
    #print(bigramList)
    unigramCount=Counter(unigramList) #aggregate all unigrams
    bigramCount=Counter(bigramList) #aggregate all bigrams
    #print(unigramCount)
    #print(bigramCount)
    return unigramCount, bigramCount
unigramCount, bigramCount = calculateCounts()

## Calculate probabilities of all candidate within given edit distance for an error word

In [10]:
def calculateBigramProbability(err_word, med):
    uniLambda = 0.4
    biLambda = 0.6
    candidateWordProbabilityDict=dict()
    candidateWords = getCandidateWords(err_word, med)
    prev_word = misspelledPrevious[err_word]
    for word in candidateWords:
        unigramProbability=unigramCount[(word,)]/len(unigramCount)
        if bigramCount[(prev_word, word)]!=0:
            bigramProbability=bigramCount[(prev_word,word)]/unigramCount[(prev_word),]
        else:
            bigramProbability=0
        candidateWordProbability=unigramProbability*uniLambda+bigramProbability*biLambda
        if candidateWordProbability != 0:
            candidateWordProbabilityDict[word] = candidateWordProbability
    return candidateWordProbabilityDict


### Find probabilies within given edit distance and range of possible candidates to correct word

In [11]:
def getWordProbabilities(err_word, correct_word, med, top=10):
    predictedWordsProbabilityDict = calculateBigramProbability(err_word, med)
    data =  sorted(predictedWordsProbabilityDict.items(), key=optr.itemgetter(1), reverse=True)[0:top]
    foundIndex = -1
    for idx, candidate_word in enumerate(data):
        if correct_word in candidate_word[0]:
            foundIndex = idx
            break
    return data, foundIndex


#### Testing cases

In [12]:
#For testing purposes
err_word = 'اادہ'
correct_word = 'اعادہ'
med = 3
test = getCandidateWords(err_word, med)
for idx, word in enumerate(test):
    if word == correct_word:
        print (idx, 'yes, in candidates')
data, fi = getWordProbabilities(err_word, correct_word, med, top=20)
if fi == -1:
    print('not found in probability dictionary, unigram, bigram probabilities 0')
else:
    for idx, word in enumerate(data):
        if word[0] == correct_word:
            print (idx, 'yes, in dictionary')

738 yes, in candidates
not found in probability dictionary, unigram, bigram probabilities 0


# Final Report

In [13]:
%%time
found_idx=list()

for idx, word in enumerate(misspelledWords):
    print('Word index:', idx, '\t\t', 'False word:', word, '\t', 'True word:', correctSpelledWords[idx], '\t', 'Previous word:', misspelledPrevious[word], '\n')
    med = 2
    predictedWordProbabilityDict, foundIndex = getWordProbabilities(word, correctSpelledWords[idx], med)
    if foundIndex == -1:
        med += 1
        predictedWordProbabilityDict, foundIndex = getWordProbabilities(word, correctSpelledWords[idx], med)
    if foundIndex == -1:
        print ('True word not found in top 10', med, 'edit distance candidate words\n')
    else:
        found_idx.append(idx)
        print ('True word found in top 10 predicted', med, 'edit distance candidates with bigram probability:', predictedWordProbabilityDict[foundIndex][1], '\n')
    print ('Candidate Index\t\tCandidate word\t\tProbability')
    for idx1, candidate_word in enumerate(predictedWordProbabilityDict):
        print_str = str(str(idx1) + '\t\t\t' + candidate_word[0]) + '\t\t\t' + str(candidate_word[1])
        if foundIndex == idx1:
            print_str =  '\x1b[0;30;43m'+'\33[1m'+print_str+' ----> TRUE WORD'+'\x1b[0m'
        print (str(print_str))
    print ('----------------------------------------------------------------------------------------------------------------\n')

print('Following are the word indices of words whose correct word was found in top 10 candidates:\n', found_idx, '\n')
print('Total misspelled words whose correct word was found', len(found_idx), '\n')

Word index: 0 		 False word: ملتن 	 True word: ملتان 	 Previous word: ڈی 

True word found in top 10 predicted 2 edit distance candidates with bigram probabilty: 0.0005592450192240475 

Candidate Index		Candidate word		Probability
0			ملتان			0.0005592450192240475 ----> TRUE WORD
1			ملتی			0.00020971688220901784
2			ملین			0.00020971688220901784
3			ملتے			6.990562740300593e-05
----------------------------------------------------------------------------------------------------------------

Word index: 1 		 False word: ہلک 	 True word: ہلاک 	 Previous word: گرد 

True word found in top 10 predicted 2 edit distance candidates with bigram probabilty: 0.2037049982523593 

Candidate Index		Candidate word		Probability
0			ہلاک			0.2037049982523593 ----> TRUE WORD
1			ملک			0.0023767913317022022
2			ہیک			0.00020971688220901784
3			ہلکی			0.00013981125480601187
4			لک			6.990562740300593e-05
5			ہلکا			6.990562740300593e-05
6			ہل			6.990562740300593e-05
-------------------------------------

True word found in top 10 predicted 2 edit distance candidates with bigram probabilty: 0.001003916418019933 

Candidate Index		Candidate word		Probability
0			خطاب			0.001003916418019933 ----> TRUE WORD
1			کتاب			0.00020971688220901784
----------------------------------------------------------------------------------------------------------------

Word index: 19 		 False word: کطاب 	 True word: خطاب 	 Previous word: کے 

True word found in top 10 predicted 2 edit distance candidates with bigram probabilty: 0.001003916418019933 

Candidate Index		Candidate word		Probability
0			خطاب			0.001003916418019933 ----> TRUE WORD
1			کتاب			0.00020971688220901784
----------------------------------------------------------------------------------------------------------------

Word index: 20 		 False word: کطاب 	 True word: خطاب 	 Previous word: کے 

True word found in top 10 predicted 2 edit distance candidates with bigram probabilty: 0.001003916418019933 

Candidate Index		Candidate word		Proba

True word found in top 10 predicted 2 edit distance candidates with bigram probabilty: 0.007887130152072479 

Candidate Index		Candidate word		Probability
0			اضافہ			0.007887130152072479 ----> TRUE WORD
----------------------------------------------------------------------------------------------------------------

Word index: 36 		 False word: فتس 	 True word: فیس 	 Previous word: کی 

True word found in top 10 predicted 2 edit distance candidates with bigram probabilty: 0.0020993725021133903 

Candidate Index		Candidate word		Probability
0			فیس			0.0020993725021133903 ----> TRUE WORD
1			فتح			0.0006652053003401624
----------------------------------------------------------------------------------------------------------------

Word index: 37 		 False word: اصافہ 	 True word: اضافہ 	 Previous word: میں 

True word found in top 10 predicted 2 edit distance candidates with bigram probabilty: 0.007887130152072479 

Candidate Index		Candidate word		Probability
0			اضافہ			0.007887130152

True word found in top 10 predicted 2 edit distance candidates with bigram probabilty: 0.00013981125480601187 

Candidate Index		Candidate word		Probability
0			مندر			0.00020971688220901784
1			منظر			0.00013981125480601187 ----> TRUE WORD
2			منزل			0.00013981125480601187
----------------------------------------------------------------------------------------------------------------

Word index: 53 		 False word: امجرم 	 True word: مجرم 	 Previous word: طاقتور 

True word found in top 10 predicted 2 edit distance candidates with bigram probabilty: 0.00027962250961202374 

Candidate Index		Candidate word		Probability
0			مجرم			0.00027962250961202374 ----> TRUE WORD
----------------------------------------------------------------------------------------------------------------

Word index: 54 		 False word: افرا 	 True word: افراد 	 Previous word: شوقین 

True word found in top 10 predicted 2 edit distance candidates with bigram probabilty: 0.006431317721076546 

Candidate Index		Cand

In [14]:
elapsedTime=time.time()-startTime
print('Total execution time:', elapsedTime)

Total execution time: 52.746370792388916
